In [4]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px


import dash
from dash import dcc
from dash import html

import plotly.graph_objects as go
from plotly.subplots import make_subplots

from datetime import datetime
# potrzebne jest zaimportowanie dodatkowych klas
from dash.dependencies import Input, Output


external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']


In [5]:
# Przygotowanie danych

# wczytanie tabeli z lekarzami
lekarze_part1 = pd.read_csv('physicians.csv')
lekarze_part2 = pd.read_csv('physicians2.csv')
lekarze_df = pd.concat([lekarze_part1, lekarze_part2])

# wczytanie tabeli z adresami
adresy_part1 = pd.read_csv('addresses.csv')
adresy_part2 = pd.read_csv('addresses2.csv')
adresy_df = pd.concat([adresy_part1, adresy_part2])


# uzupełnienie tabeli z lekaarzami o zgrabniejsze zmienne
# zróbmy liczby z opinion count
lekarze_df['opinionCount'] = lekarze_df['opinionCount'].str.extract('(\d*)').astype(int)

# dodajmy zmienną czy udziela porad online
lekarze_df['ifOnline'] = lekarze_df['online'].astype(bool)


# stworzenie tabeli z agregatami nt specjalizacji
specialization_df = lekarze_df.groupby('specializations')[['specializations', 'rating', 'opinionCount', 'ifOnline']].agg(
    {'specializations':'count', 'rating':'mean', 'opinionCount': 'mean', 'ifOnline': 'sum'})
specialization_df.rename(columns={'specializations': 'liczba_lekarzy'}, inplace=True) # nadanie nazwy kolumnie z liczbą lekarzy
specialization_df.sort_values(by='liczba_lekarzy', ascending=False, inplace=True) # posortowanie wg liczby lekarzy

# stworzenie tabeli z agregatami nt specjalizacji dla lekarzy z poradami online
# optymalniej byłoby to filtrowanie zrobić w kodzie samego dashboardu, do poprawienia w przyszłości 
lekarze_online = lekarze_df[lekarze_df['ifOnline']]
specialization_online_df = lekarze_online.groupby('specializations')[['specializations', 'rating', 'opinionCount', 'ifOnline']].agg(
    {'specializations':'count', 'rating':'mean', 'opinionCount': 'mean', 'ifOnline': 'sum'})
specialization_online_df.rename(columns={'specializations': 'liczba_lekarzy'}, inplace=True) # nadanie nazwy kolumnie z liczbą lekarzy
specialization_online_df.sort_values(by='liczba_lekarzy', ascending=False, inplace=True) # posortowanie wg liczby lekarzy
specialization_online_df


# stworzenie tabeli łączącej lekarzy i adresy:
lekarze_adresy_df = pd.merge(lekarze_df, adresy_df, on= 'physicianID')

In [6]:
 

# kolor tła wykresów
back_color1 = 'rgba(209, 229, 235, 0.1)'
back_color2 = 'rgba(209, 229, 235, 0.2)'

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div([
    
    html.Div([
        dcc.Markdown('''### Lekarze i inni specjaliści na stronie **Znany Lekarz**''')
    ], style={'display': 'inline-block', 'margin-left': '1vw', 'margin-right': '2vw', 'height':'50px',
              'background-color': back_color2, 'padding-top': '5px', 'padding-right': '20px', 'padding-bottom': '5px',
                    'padding-left': '5px'}),
    
    html.Div([
        html.Label('Wybierz specjalizację'),
        dcc.Dropdown(id='dropdown1', value='Wszystkie',
            options=[{'label': 'Wszystkie', 'value':'Wszystkie'}] + [{'label':spec, 'value':spec} for spec in specialization_df.index],
                    style={'width':'300px'})
    ], style={'display': 'inline-block', 'margin-left': '1vw', 'vertical-align': 'top', 'background-color': back_color2}),
    
    html.Div([
        html.Label('Czy oferuje porady online'),
        dcc.Dropdown(id='dropdown2', value='Wszyscy',
            options=[{'label': 'Wszyscy', 'value':'Wszyscy'},
                     {'label': 'Tylko z możliwością porady online','value':'Tylko z możliwością porady online'}],
                    style={'width':'300px'})        
        ], style={'display': 'inline-block', 'margin-left': '1vw', 'vertical-align': 'top', 'background-color': back_color2}),
    
    html.Div([
        dcc.Markdown('''      ```Stan na grudzień 2022 r.```''')
    ], style={'display': 'inline-block', 'margin-left': '1vw', 'margin-right': '2vw', 'height':'50px',
              'vertical-align': 'top', 'color':'darkblue', 'fontSize':12, 'text-align': 'right', 'width': '300px'}),
    
    html.Div(id='sekcja-wizualizacje')
    
])



# część z aktualizującymi się wizualizacjami:
@app.callback(Output('sekcja-wizualizacje', 'children'), [Input('dropdown1', 'value'), Input('dropdown2', 'value')])
def wizualizacje(spec, if_online):
    
    # przefitrowywanie zbiorów

    # przez możliwość porad online 
    if if_online == 'Tylko z możliwością porady online':
        
        spec_df = specialization_online_df
        lek_adr = lekarze_adresy_df[lekarze_adresy_df['ifOnline']]
    else:
        spec_df = specialization_df
        lek_adr = lekarze_adresy_df
    
    # przez specjalizacje
    if spec == 'Wszystkie':
        df = spec_df
        df_lek_adr = lek_adr
    else:
        df = spec_df[spec_df.index == spec]
        df_lek_adr = lek_adr[lek_adr.specializations == spec]
    
    df_opinions_sorted = df.sort_values(by='opinionCount', ascending=False)
    

    
    # 1 wykres opinii    
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    fig.add_trace(
        go.Bar(x=df_opinions_sorted.index, y=df_opinions_sorted.opinionCount, name="Liczba opinii", offsetgroup=1),
        secondary_y=False,
    )
    fig.add_trace(
        go.Bar(x=df_opinions_sorted.index, y=df_opinions_sorted.rating, name="Średnia ocena", offsetgroup=2),
        secondary_y=True,
    )
    fig.update_xaxes(title_text="Specjalizacje")
    fig.update_yaxes(title_text="Liczba opinii", secondary_y=False)
    fig.update_yaxes(title_text="Średnia ocena", secondary_y=True)
    fig.update_yaxes(range=[4.2,6], secondary_y=True)
    fig.update_layout({'paper_bgcolor': back_color1, 'title':'Liczba opinii i średnia ocen wg. specjalizacji'})
    
    # 2 mapa województw
    # wczytanie granic województw    
    woj_granice = gpd.read_file('wojewodztwa.geojson')
    woj_granice=woj_granice.set_index('nazwa')
    
    # wczytanie ludności GUS
    ludnosc_gus = pd.read_csv('LUDN_2137_CTAB_20230305013744.csv', encoding='windows-1250', sep=';')
    
    # zagregowanie liczby lekarzy
    lekarze_woj = df_lek_adr.groupby('region')['physicianID'].nunique('physicianID')
    lekarze_woj = lekarze_woj.reset_index()
    
    # połączenie lekarzy i ludności
    lekarze_ludnosc = pd.merge(ludnosc_gus, lekarze_woj, left_on='Nazwa', right_on='region')
    lekarze_ludnosc['Specjalistów na 100tys'] = lekarze_ludnosc.physicianID / lekarze_ludnosc.ludnosc * 100000
    lekarze_ludnosc['Specjalistów na 100tys'] = lekarze_ludnosc['Specjalistów na 100tys'].round(2)
    
    # stworzenie mapy województw
    mapa = px.choropleth(data_frame=lekarze_ludnosc, geojson=woj_granice, locations="region", color="Specjalistów na 100tys",
                         color_continuous_scale="Blues", projection="mercator")
    mapa.update_geos(fitbounds="locations", visible=False)
    mapa.update_layout({'paper_bgcolor': back_color1, 'plot_bgcolor': 'rgba(0,0,0,0)'})
    mapa.update_layout(geo=dict(bgcolor= 'rgba(0,0,0,0)'), title='Profile na 100 tys mieszkańców wg województw')
    
    # 3 heatmapa    
    heatmap = px.density_mapbox(df_lek_adr, lat='latitude', lon='longitude', radius=3,
                        center=dict(lat=52, lon=19), zoom=4.6,
                        mapbox_style='carto-positron'
                                #, color_continuous_scale='Blues'
                               )
    heatmap.update_layout({'paper_bgcolor': back_color1, 'title': 'Miejsca udzielania świadczeń',
                           'coloraxis_showscale':False})
    
    # 4 wykres z liczebnościami
    liczebnosci = go.Figure(data=[go.Bar(x=df.index, y=df.liczba_lekarzy)],
                           layout=go.Layout(paper_bgcolor = back_color1, title='Liczba profili wg. specjalizacji',
                                           height=420))
    liczebnosci.update_xaxes(tickangle=45)


    styl_tekstu = {'fontSize':18, 'background-color': back_color1, 'text-align':'center', 'display': 'inline-block',
                  'margin-right': '1vw', 'padding-top': '5px', 'padding-right': '20px', 'padding-bottom': '5px',
                    'padding-left': '20px'}
    
        
    return [
            html.Div([
                html.Div([
                    html.Div([
                        dcc.Markdown(f'''Zweryfikownych profili w Polsce:   **{df_lek_adr.physicianID.nunique()}**''')
                    ], style=styl_tekstu),

                    html.Div([
                        dcc.Markdown(f'''Zweryfikowanych profili na 100 tys. ludności w Polsce:
                          **{round(df_lek_adr.physicianID.nunique()/ 37907704 * 100000, 2)}**''')
                    ], style=styl_tekstu)
                ]),
                dcc.Graph(figure=liczebnosci,
                         style={'display': 'inline-block', 'vertical-align': 'top', 'margin-right': '2vw', 'margin-top': '0.5vw',
                               'width':'1000px'})                
            ], style={'display': 'inline-block', 'vertical-align': 'top', 'margin-top': '0.5vw', 'height':'560px'}),

            dcc.Graph(figure=mapa,
                     style={'display': 'inline-block', 'vertical-align': 'top', 'margin-right': '2vw', 'margin-top': '0.5vw',
                           'height':'350px'}),
            dcc.Graph(figure=fig,
                     style={'display': 'inline-block', 'vertical-align': 'top', 'margin-right': '2vw', 'margin-top': '1vw',
                           'width':'1000px', 'position': 'relative', 'top': '-100px', 'height':'400px'}),
            dcc.Graph(figure=heatmap,
                        style={'display': 'inline-block',  'horizontal-align': 'right', 'margin-right': '2vw', 'margin-top': '1vw',
                               'position': 'relative', 'top': '-221px', 'height':'540px'})
            
           ]



if __name__ == '__main__':
    app.run_server()


Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [07/Mar/2023 17:11:53] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [07/Mar/2023 17:11:54] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [07/Mar/2023 17:11:54] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [07/Mar/2023 17:11:54] "GET /_dash-component-suites/dash/dcc/async-markdown.js HTTP/1.1" 304 -
127.0.0.1 - - [07/Mar/2023 17:11:54] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 304 -
127.0.0.1 - - [07/Mar/2023 17:11:54] "GET /_dash-component-suites/dash/dcc/async-highlight.js HTTP/1.1" 304 -
127.0.0.1 - - [07/Mar/2023 17:11:55] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [07/Mar/2023 17:11:55] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [07/Mar/2023 17:11:55] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -
